In [1]:
%load_ext autoreload
%autoreload 2

# Spark Preprocessing

In [1]:
from pyspark.sql import SparkSession, functions as F

spark = SparkSession.builder.getOrCreate()
df = spark.read.parquet("../data/processed/enwiki-meta-compact")
df.createOrReplaceTempView("enwiki")

In [2]:
def get_edges_blocked():
    query = """
    with subset as (
        SELECT
            concat(year, '-', quarter) as period,
            dayofyear(timestamp) as doy,
            article_id,
            cast(user_id as int) as user_id,
            textdata
        FROM
            enwiki
    )
    -- collect the weighted edge-list
    SELECT
        user_id,
        article_id,
        doy,
        sum(textdata) as word_count,
        count(*) as num_edits 
    FROM
        subset
    WHERE
        subset.user_id is not null
    GROUP BY 1, 2, 3
    """

    return spark.sql(query)


def project_common_neighbors_by_day(edges, threshold):
    edges.createOrReplaceTempView("edges")
    query = """
    with unimodal_projection as (
        SELECT
            t1.user_id as e1,
            t2.user_id as e2,
            count(*) as shared_articles
        FROM edges t1
        JOIN edges t2 
        ON t1.article_id = t2.article_id AND t1.doy = t2.doy
        WHERE t1.user_id < t2.user_id
        GROUP BY 1, 2
    )

    SELECT e1, e2, shared_articles
    FROM unimodal_projection
    WHERE shared_articles > {}
    """.format(threshold)
    return spark.sql(query)
    

edge_blocked = get_edges_blocked("2007-1")
elist_day = project_common_neighbors_by_day(edge_blocked, 0)
elist_day.cache()
elist_day.count()

1825594

In [13]:
df.selectExpr("date_format(timestamp, 'yyyy-MM-dd')").show()

+----------------------------------+
|date_format(timestamp, yyyy-MM-dd)|
+----------------------------------+
|                        2007-07-19|
|                        2007-07-19|
|                        2007-07-19|
|                        2007-07-19|
|                        2007-07-19|
|                        2007-07-19|
|                        2007-07-19|
|                        2007-07-19|
|                        2007-07-19|
|                        2007-07-19|
|                        2007-07-19|
|                        2007-07-19|
|                        2007-07-19|
|                        2007-07-19|
|                        2007-07-19|
|                        2007-07-19|
|                        2007-07-19|
|                        2007-07-19|
|                        2007-07-19|
|                        2007-07-19|
+----------------------------------+
only showing top 20 rows



In [15]:
query = """
with subset as (
    SELECT
        date_format(timestamp, 'yyyy-MM-dd') as edit_date,
        article_id,
        cast(user_id as int) as user_id,
        textdata
    FROM
        enwiki
),
-- collect the weighted edge-list
bipartite as (
    SELECT
        user_id,
        article_id,
        edit_date,
        sum(log(textdata)) as word_count,
        count(*) as num_edits 
    FROM
        subset
    WHERE
        subset.user_id is not null
    GROUP BY 1, 2, 3
),
unimodal_projection as (
    SELECT
        t1.user_id as e1,
        t2.user_id as e2,
        count(*) as shared_articles
    FROM bipartite t1
    JOIN bipartite t2 
    ON t1.article_id = t2.article_id AND t1.edit_date = t2.edit_date
    WHERE t1.user_id < t2.user_id
    GROUP BY 1, 2
)

SELECT e1, e2, shared_articles
FROM unimodal_projection
"""

spark.sql(query).count()

12632948

In [18]:
query = """
with subset as (
    SELECT
        date_format(timestamp, 'yyyy-MM-dd') as edit_date,
        article_id,
        cast(user_id as int) as user_id,
        textdata
    FROM
        enwiki
),
-- collect the weighted edge-list
bipartite as (
    SELECT
        user_id,
        article_id,
        edit_date,
        sum(log(textdata)) as word_count,
        count(*) as num_edits 
    FROM
        subset
    WHERE
        user_id is not null
    GROUP BY 1, 2, 3
)

select
    article_id,
    edit_date,
    count(distinct user_id)
from bipartite
group by 1, 2
order by 3 desc
"""
spark.sql(query).show()

Py4JJavaError: An error occurred while calling o187.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 10 in stage 30.0 failed 1 times, most recent failure: Lost task 10.0 in stage 30.0 (TID 2415, localhost, executor driver): java.lang.OutOfMemoryError: Java heap space

Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1887)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1875)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1874)
	at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:48)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:1874)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleTaskSetFailed$1.apply(DAGScheduler.scala:926)
	at 